# 순환신경망 (Recurrent Neural Network : RNN )

   + 순서가 있는 데이터를 입력으로 받고, 같은 네트워크를 이용해 변화하는 입력에 대한 출력을 얻는다
   + 순환신경망(RNN)은 여러 개의 데이터가 순서대로 입력되었을 때 앞서 입력받은 데이터를 잠시 기억해 놓는 방법이다.
   + 기억된 데이타가 얼마나 중요한지를 판단하여 별도의 가중치를 줘서 다음 데이터로 넘어간다
   + 모든 입력 값에 이 작업을 순서대로 실행하므로 다음 층에서 넘어가지 전에 같은 층을 맴도는 것처럼 보여서 순환신경망이라고 부른다
    
   <img src='./imgs/rnn1.png' width='400' height='250'>
   <img src='./imgs/rnn2.png' width='400' height='250'>
   
   [출처] 모두의 딥러닝

##  1. SimpleRNN 레이어

[ 0.0, 0.1, 0.2, 0.3 ] 이라는 연속된 숫자가 주어졌을 때 0.4를 예측하도록 네트워크를 만드는 것이다.

### 시퀀스 예측 모델 정의

<img src='./imgs/rnn3.png' width='600' height='450'>

In [1]:
import tensorflow as tf
import numpy as np

# (1) 시퀀스 예측 데이터 생성
X = []    # 입력데이타
Y = []    # 출력데이타

for i in range(6): # 0부터 5까지
    
    # [0,1,2,3], [1,2,3,4] 같은 정수의 시퀀스를 만듭니다.
    lst = list(range(i,i+4)) # 0부터 4전의 3까지 list 만들기

    # 위에서 구한 시퀀스의 숫자들을 각각 10으로 나눈 다음 저장합니다.
    # SimpleRNN 에 각 타임스텝에 하나씩 숫자가 들어가기 때문에 여기서도 하나씩 분리해서 배열에 저장합니다.
    # map(f, iterable) : 함수(f)와 반복 가능한 (iterable) 자료형을 입력으로 받는다.
    # result = list(map(lambda i: i ** 2 , li)) : li 요소들을 i로 하나씩 i**2 함수로 한 결과를 다시 list로 변환
    X.append(list(map(lambda c: [c/10], lst)))

    # 정답에 해당하는 4, 5 등의 정수를 역시 위처럼 10으로 나눠서 실수로 저장합니다.
    Y.append((i+4)/10)
    
X = np.array(X)
Y = np.array(Y)

# 입력데이타(X)와 출력데이타(Y) 확인
for i in range(len(X)):
    print(X[i], '-->', Y[i])

[[0. ]
 [0.1]
 [0.2]
 [0.3]] --> 0.4
[[0.1]
 [0.2]
 [0.3]
 [0.4]] --> 0.5
[[0.2]
 [0.3]
 [0.4]
 [0.5]] --> 0.6
[[0.3]
 [0.4]
 [0.5]
 [0.6]] --> 0.7
[[0.4]
 [0.5]
 [0.6]
 [0.7]] --> 0.8
[[0.5]
 [0.6]
 [0.7]
 [0.8]] --> 0.9


In [2]:
# (2) 시퀀스 예측 모델 정의
#     출력을 위해 Dense 레이어를 뒤에 추가
#     input_shape=[timesteps, input_dim]
#     timesteps : 순환신경망이 입력에 대해 계산을 반복하는 횟수 ( 위에 보면 ) [[0. ] [0.1] [0.2] [0.3]] 으로 4번이다
#     input_dim : 입력벡터의 크기
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(units=10, return_sequences=False, input_shape=[4,1]),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 10)                120       
_________________________________________________________________
dense (Dense)                (None, 1)                 11        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


In [3]:
# (3) 네트워크 훈련 및 결과 확인
#     verbose=0 : 훈련과정에서의 출력이 나오지 않음 ( 출력 볼 필요 없을 때 )
model.fit(X, Y, epochs=100, verbose=0)
print(model.predict(X))

[[0.39153188]
 [0.5443831 ]
 [0.65469146]
 [0.72774255]
 [0.77327514]
 [0.80033207]]


[ 결과 ] 

X (학습 데이타) : [[0. ] [0.1] [0.2] [0.3]]  -->  예측값 :  0.43109146  즉 0.4와 유사하다
 

In [4]:
# (4) 학습되지 않은 시퀀스에 대한 예측 결과
print(model.predict(np.array([[[0.6],[0.7],[0.8],[0.9]]])))
print(model.predict(np.array([[[-0.1],[0.0],[0.1],[0.2]]])))

[[0.81573063]]
[[0.2041359]]


[결과]

[[[-0.1],[0.0],[0.1],[0.2]]] --> [[0.28398907]] : 원하는 출력으로 0.3을 예측하였지만
    
[[[0.6],[0.7],[0.8],[0.9]]] -->  [[0.8932602]]  : 원하는 출력으로 1.0을 예측하지 못하였다.

* 신경망에 일반화된 규칙을 학습시키는 것은 아직까지 어려운 과제 중 하나랍니다